In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F

nclasses = 43 # GTSRB as 43 classes

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 1, kernel_size=1)
        self.conv2 = nn.Conv2d(1, 29, kernel_size=5)
        self.maxpool2 = nn.MaxPool2d(3, stride=2 , ceil_mode=True)
        self.conv3 = nn.Conv2d(29, 59, kernel_size=3)
        self.maxpool3 = nn.MaxPool2d(3, stride=2 , ceil_mode=True)
        self.conv4 = nn.Conv2d(59, 74, kernel_size=3)
        self.maxpool4 = nn.MaxPool2d(3, stride=2 , ceil_mode=True)
        self.conv2_drop = nn.Dropout2d()
        self.conv3_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(1184, 300)
        self.fc2 = nn.Linear(300, nclasses)
        self.conv0_bn = nn.BatchNorm2d(3)
        self.conv1_bn = nn.BatchNorm2d(1)
        self.conv2_bn = nn.BatchNorm2d(29)
        self.conv3_bn = nn.BatchNorm2d(59)
        self.conv4_bn = nn.BatchNorm2d(74)
        self.dense1_bn = nn.BatchNorm1d(300)
    def forward(self, x):
        x =  F.relu(self.conv1_bn(self.conv1(self.conv0_bn(x))))
        x = F.relu(self.conv2_bn(self.conv2(x)))
        x = F.relu(self.conv3_bn(self.conv3( self.maxpool2(x))))
        x = F.relu(self.conv4_bn(self.conv4( self.maxpool3(x))))
        x = self.maxpool4(x)        
        x = x.view(-1, 1184)
        x = F.relu(self.fc1(x))
        x = self.dense1_bn(x)
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        x = F.dropout(x, training=self.training)
        return F.log_softmax(x, dim=1)
        
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class MicronNetBF(nn.Module):
#     def __init__(self, num_classes: int = 43):
#         super().__init__()

#         # stage 1
#         self.conv1 = nn.Sequential(
#             nn.Conv2d(3, 3, kernel_size=1, bias=False),
#             nn.BatchNorm2d(3),
#             nn.ReLU(inplace=True)
#         )

#         # stage 2
#         self.conv2 = nn.Sequential(
#             nn.Conv2d(3, 29, kernel_size=3, bias=False),
#             nn.BatchNorm2d(29),
#             nn.ReLU(inplace=True)
#         )

#         # stage 3
#         self.conv3 = nn.Sequential(
#             nn.Conv2d(29, 40, kernel_size=3, bias=False),
#             nn.BatchNorm2d(40),
#             nn.ReLU(inplace=True)
#         )

#         # stage 4
#         self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2)

#         self.conv4 = nn.Sequential(
#             nn.Conv2d(40, 59, kernel_size=3, bias=False),
#             nn.BatchNorm2d(59),
#             nn.ReLU(inplace=True)
#         )

#         # stage 5
#         self.pool2 = nn.MaxPool2d(kernel_size=3, stride=2)

#         self.conv5 = nn.Sequential(
#             nn.Conv2d(59, 74, kernel_size=3, bias=False),
#             nn.BatchNorm2d(74),
#             nn.ReLU(inplace=True)
#         )

#         # stage 6
#         self.pool3 = nn.MaxPool2d(kernel_size=3, stride=2)

#         # fully-connected head
#         self.fc1 = nn.Sequential(
#             nn.Linear(74 * 4 * 4, 300, bias=False),
#             nn.BatchNorm1d(300),
#             nn.ReLU(inplace=True)
#         )
#         self.fc2 = nn.Linear(300, num_classes)

#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.conv2(x)
#         x = self.conv3(x)
#         x = self.pool1(x)

#         x = self.conv4(x)
#         x = self.pool2(x)

#         x = self.conv5(x)
#         x = self.pool3(x)

#         x = torch.flatten(x, 1)
#         x = self.fc1(x)
#         x = self.fc2(x)
#         return x


In [19]:
from __future__ import print_function
import argparse
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable


batch_size =50                                                                                                                                                                                     
epochs = 70                                                                                                                                                                                                                                                                                                                                                                                 
seed = 1                                                                                                                                                                                           
log_interval=180                                                                                                                                                                                   
data = "data"                                                                                                                                                                                                                                                                                                                                                                                          
torch.manual_seed(1)                                                                                                                                                                               
lr = 0.007                                                                                                                                                                                          
momentum = 0.9                                                                                                                                                                                     
decay =  0.9996                                                                                                                                                                                        
step = 1000                                                                                                                                                                                        
l2_norm = 0.00001  
cuda = True
resume = False
# These may change as described in paper

In [20]:
# from data import initialize_data, data_transforms 
# initialize_data("data") # extracts the zip files, makes a validation set
data_transforms = transforms.Compose([
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
])

train_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(data + '/train_images',
                         transform=data_transforms),
    batch_size=batch_size, shuffle=True, num_workers=1)




val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(data + '/val_images',
                         transform=data_transforms),
    batch_size=batch_size, shuffle=False, num_workers=1)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = Net()
if  cuda: 
    model.to(device)

if resume :
    state_dict = torch.load("model_28.pth")
    model.load_state_dict(state_dict) 



In [21]:
           
def validation():
    model.eval()
    validation_loss = 0
    correct = 0
    for data, target in val_loader:
        data, target = Variable(data), Variable(target)
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        validation_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        validation_loss, correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))
    return validation_loss

def train(epoch , train_loader):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target).cuda()
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                # 100. * batch_idx / len(train_loader), loss.data[0]))
                100. * batch_idx / len(train_loader), loss.item()))


In [23]:
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=l2_norm, nesterov=True)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=decay)
best_val_loss = float('inf')

for epoch in range(1, epochs):
    train(epoch, train_loader)
    val_loss = validation()
    if epoch % step:
        scheduler.step()
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        model_file = 'best_model.pth'
        torch.save(model.state_dict(), model_file)
        print('\nBest model updated and saved to ' + model_file)

Train Epoch: 1 [0/35339 (0%)]	Loss: 1.615807
Train Epoch: 1 [9000/35339 (25%)]	Loss: 1.368937
Train Epoch: 1 [18000/35339 (51%)]	Loss: 1.637118
Train Epoch: 1 [27000/35339 (76%)]	Loss: 1.612476

Validation set: Average loss: 1023.3083, Accuracy: 3725/3870 (96%)


Best model updated and saved to best_model.pth
Train Epoch: 2 [0/35339 (0%)]	Loss: 1.735775
Train Epoch: 2 [9000/35339 (25%)]	Loss: 1.315583
Train Epoch: 2 [18000/35339 (51%)]	Loss: 1.682572
Train Epoch: 2 [27000/35339 (76%)]	Loss: 1.351376

Validation set: Average loss: 2447.1482, Accuracy: 3398/3870 (88%)

Train Epoch: 3 [0/35339 (0%)]	Loss: 1.446374
Train Epoch: 3 [9000/35339 (25%)]	Loss: 1.277853
Train Epoch: 3 [18000/35339 (51%)]	Loss: 1.933936
Train Epoch: 3 [27000/35339 (76%)]	Loss: 1.579352

Validation set: Average loss: 858.1703, Accuracy: 3723/3870 (96%)


Best model updated and saved to best_model.pth
Train Epoch: 4 [0/35339 (0%)]	Loss: 1.508081
Train Epoch: 4 [9000/35339 (25%)]	Loss: 1.488083
Train Epoch: 4 [18000/